# Data pipeline double check

* check various nifti / tensors / normalizations schemes are not affecting quality of data

In [1]:
### Base ###
import os
import sys
import csv
import pickle
import shutil
from argparse import Namespace
import pandas as pd
import numpy as np 
import torch 
import torch.nn as nn
from torch.optim import Adam
import nibabel as nib
from nilearn.image import resample_img
import fnmatch
import itertools
import math
from sklearn.decomposition import PCA
import PIL.Image as pimg
import pytorch_lightning as pl
import torch.utils.data as data_utils

### Visualization ###
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.mlab as mlab
from matplotlib import rc
rc('text', usetex=True)
rc('font', **{'family':'serif','serif':['Palatino']})

parent = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
sys.path.insert(0, parent)
os.chdir(parent)
print('Setting root path to : {}'.format(parent))

from src.support.base_miccai import *
from src.in_out.datasets_miccai import ZeroOneT13DDataset

/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/network/lustre/dtlake01

Setting root path to : /network/lustre/dtlake01/aramis/users/paul.vernhet/Scripts/Ongoing/MICCAI2020/deepshape


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/torchvision/extension.py:11: ResourceWarning: unclosed file <_io.BufferedReader name='/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/torchvision/_C.so'>
  _, path, _ = imp.find_module("_C", [lib_dir])


In [2]:
# PATHS :
HOME_PATH = '/network/lustre/dtlake01/aramis/users/paul.vernhet'
# Original data
ORIGINAL_NIFTI_PATH = os.path.join(HOME_PATH, 'Data/MICCAI_dataset/2_datasets/1_brats_2019')
ORIGINAL_BIDS_PATH = os.path.join(HOME_PATH, 'Data/MICCAI_dataset/4_segmentations/BIDS')
SEGMENTED_ORIGINAL_BIDS_PATH = os.path.join(HOME_PATH, 'Data/MICCAI_dataset/4_segmentations/CAPS')
# Preprocessed datas
PREPROCESSED_NIFTI_PATH = os.path.join(HOME_PATH, 'Data/MICCAI_dataset/2_datasets/2_t1ce_normalized')
TENSORS_PATH = os.path.join(HOME_PATH, 'Data/MICCAI_dataset/3_tensors3d/2_t1ce_normalized/0_reduction')
# Output dirs
OUTPUT_DIR = os.path.join(HOME_PATH, 'Results/MICCAI/pipeline_checks')
assert os.path.exists(ORIGINAL_NIFTI_PATH)
assert os.path.exists(ORIGINAL_BIDS_PATH)
assert os.path.exists(SEGMENTED_ORIGINAL_BIDS_PATH)
assert os.path.exists(PREPROCESSED_NIFTI_PATH)
assert os.path.exists(TENSORS_PATH)

In [3]:
num_gpu = 0

torch.manual_seed(222)
np.random.seed(222)
DEVICE = torch.device('cuda')
torch.cuda.set_device(num_gpu)
torch.cuda.manual_seed(222)

Get 3d data loaders initialized :

In [4]:
# Data loaders
dataset_train = ZeroOneT13DDataset(os.path.join(TENSORS_PATH, 'train'), 64,
                                           reduction=1, init_seed=256,
                                           check_endswith='pt', is_half=False)
train_loader = data_utils.DataLoader(dataset_train, batch_size=32, shuffle=True)
np_affine = dataset_train.affine.numpy()


def crop_and_save_nifti(input_path, output_path, ref_crop):
    input_img = nib.load(input_path)
    cropped_img = resample_img(input_img, target_affine=ref_crop.affine)  # resample to reference affine | crop and save accordingly
    cropped_img = nib.Nifti1Image(input_img.get_data()[40:-40, 24:-24, 14:-13], cropped_img.affine)  
    nib.save(cropped_img, output_path)

>> Creating dataset with 64 files (from 335 available)
>> Affine matrix was detected :
 [[ -1.   0.   0.  -0.]
 [  0.  -1.   0. 239.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]]


In [5]:
# dataframes
df_namemapping = pd.read_csv(os.path.join(ORIGINAL_NIFTI_PATH, '1_training', 'name_mapping.csv'))
df_survival = pd.read_csv(os.path.join(ORIGINAL_NIFTI_PATH, '1_training', 'survival_data.csv'))
df_segmented_links = pd.read_csv(os.path.join(ORIGINAL_BIDS_PATH, '1_training', 'table_links.csv'))

Save in same folder : 
* original nifti
* tissue segmented original nifti (if available)
* preprocessed nifti (normalized)
* preprocessed nifti from tensor copy [input before 0-1 normalization]
* preprocessed nifti from tensor copy + 0-1 normalization + reverse normalization | should be perfectly identical to previous line ...

In [6]:
do_tensor_slicing = True

for i in range(5):
    
    # ==========================
    # Tensor
    intensity_tensor, filename = dataset_train.__getitem__(i)
    reference_crop = nib.Nifti1Image(gpu_numpy_detach(intensity_tensor.squeeze()), np_affine)
    
    # ==========================
    # Name
    print('>> Subject :', filename)
    split_name = filename.split('_')
    which_tumour = split_name[0].upper()
    main_name = 'BraTS19_' + '_'.join(split_name[1:-1])
    infos = {}
    if len(df_survival[df_survival['BraTS19ID'] == main_name]):
        
        row = df_survival[df_survival['BraTS19ID'] == main_name][['Age', 'Survival']].values[0]
        infos['age'] = str(row[0])
        infos['survival'] = str(row[1])
    else:
        infos['age'] = 'NA'
        infos['survival'] = 'NA'
    
    # ==========================
    # Preprocessed
    t1ce_preprocessed_nifti = os.path.join(PREPROCESSED_NIFTI_PATH, 'train', filename.split('.')[0] + '.nii.gz')
    assert os.path.exists(t1ce_preprocessed_nifti), t1ce_preprocessed_nifti
    
    # ==========================
    # Segmentations
    dir_t1ce_original_nifti = os.path.join(ORIGINAL_NIFTI_PATH , '1_training', which_tumour, main_name)    # '{_t1ce, _seg}.nii.gz'
    assert os.path.exists(dir_t1ce_original_nifti), dir_t1ce_original_nifti
    t1ce_original_nifti = os.path.join(dir_t1ce_original_nifti, main_name + '_t1ce.nii.gz')
    assert os.path.exists(t1ce_original_nifti), t1ce_original_nifti
    tumorsegmented_original_nifti = os.path.join(dir_t1ce_original_nifti, main_name + '_seg.nii.gz')
    assert os.path.exists(tumorsegmented_original_nifti), tumorsegmented_original_nifti
    
    t1ce_original_bids = df_segmented_links[df_segmented_links['src_file'] == t1ce_original_nifti]['dest_file'].values[0]
    assert os.path.exists(t1ce_original_bids), t1ce_original_bids
    t1ce_original_centered_bids = t1ce_original_bids.replace('1_training', '1_training_centered')
    assert os.path.exists(t1ce_original_centered_bids), t1ce_original_centered_bids

    temp_centered_segmented_bids = t1ce_original_centered_bids.replace('BIDS',
                                                                       'CAPS').replace('1_training_centered',
                                                                                                 '1_training_centered/subjects')
    sessionhead_t1ce_centered_segmented_bids = os.path.dirname(os.path.dirname(temp_centered_segmented_bids))
    
    # ==========================
    # Check if segmentation exists
    if not os.path.exists(sessionhead_t1ce_centered_segmented_bids):
        print('>> subject segmentation not available | skipping to next subject')
        break
    
    # if so, get infos
    dirhead_t1ce_centered_segmented_bids = os.path.join(sessionhead_t1ce_centered_segmented_bids, 't1/spm/segmentation')
    assert os.path.exists(dirhead_t1ce_centered_segmented_bids), dirhead_t1ce_centered_segmented_bids

    csf_filename = t1ce_original_centered_bids.split('/')[-1].split('.')[0] + '_segm-csf_probability.nii.gz'
    whitematter_filename = t1ce_original_centered_bids.split('/')[-1].split('.')[0] + '_segm-whitematter_probability.nii.gz'
    graymatter_filename = t1ce_original_centered_bids.split('/')[-1].split('.')[0] + '_segm-graymatter_probability.nii.gz'
    t1ce_csf_nifti = os.path.join(dirhead_t1ce_centered_segmented_bids, 'native_space', csf_filename)
    t1ce_whitematter_nifti = os.path.join(dirhead_t1ce_centered_segmented_bids, 'native_space', whitematter_filename)
    t1ce_graymatter_nifti = os.path.join(dirhead_t1ce_centered_segmented_bids, 'native_space', graymatter_filename)
    assert os.path.exists(t1ce_csf_nifti), t1ce_csf_nifti
    assert os.path.exists(t1ce_whitematter_nifti), t1ce_csf_nifti
    assert os.path.exists(t1ce_graymatter_nifti), t1ce_csf_nifti
    
    # ==========================
    # Save data as nifti files | rescale data to match tensor narrowing 
    
    saving_dir = os.path.join(OUTPUT_DIR, filename)
    os.makedirs(saving_dir, exist_ok=True)

    with open(os.path.join(saving_dir, 'dict.pickle'), 'wb') as handle:
        pickle.dump(infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    if do_tensor_slicing:
        crop_and_save_nifti(t1ce_original_nifti, os.path.join(saving_dir, '{}_original.nii.gz'.format(filename)), reference_crop)
        crop_and_save_nifti(tumorsegmented_original_nifti, os.path.join(saving_dir, '{}_tumorseg.nii.gz'.format(filename)), reference_crop)
        crop_and_save_nifti(t1ce_csf_nifti, os.path.join(saving_dir, '{}_csf.nii.gz'.format(filename)), reference_crop)
        crop_and_save_nifti(t1ce_whitematter_nifti, os.path.join(saving_dir, '{}_whitematter.nii.gz'.format(filename)), reference_crop)
        crop_and_save_nifti(t1ce_graymatter_nifti, os.path.join(saving_dir, '{}_graymatter.nii.gz'.format(filename)), reference_crop)
        crop_and_save_nifti(t1ce_preprocessed_nifti, os.path.join(saving_dir, '{}_preprocessed.nii.gz'.format(filename)), reference_crop)
    else:
        _ = shutil.copy(t1ce_original_nifti, os.path.join(saving_dir, '{}_original.nii.gz'.format(filename)))
        _ = shutil.copy(tumorsegmented_original_nifti, os.path.join(saving_dir, '{}_tumorseg.nii.gz'.format(filename)))
        _ = shutil.copy(t1ce_csf_nifti, os.path.join(saving_dir, '{}_csf.nii.gz'.format(filename)))
        _ = shutil.copy(t1ce_whitematter_nifti, os.path.join(saving_dir, '{}_whitematter.nii.gz'.format(filename)))
        _ = shutil.copy(t1ce_graymatter_nifti, os.path.join(saving_dir, '{}_graymatter.nii.gz'.format(filename)))

        _ = shutil.copy(t1ce_preprocessed_nifti, os.path.join(saving_dir, '{}_preprocessed.nii.gz'.format(filename)))
        
    nib.save(reference_crop, os.path.join(saving_dir, '{}_fromtensor.nii.gz'.format(filename)))

>> Subject : lgg_TCIA10_640_1_t1ce


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/_utils/niimg.py:116: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dtype = _get_target_dtype(niimg.get_data().dtype, dtype)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/image/resampling.py:442: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()
/network/lustre/dtlake0

>> Subject : lgg_TCIA09_141_1_t1ce


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/_utils/niimg.py:116: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dtype = _get_target_dtype(niimg.get_data().dtype, dtype)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/image/resampling.py:442: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()
/network/lustre/dtlake0

>> Subject : lgg_TCIA09_451_1_t1ce


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/_utils/niimg.py:116: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dtype = _get_target_dtype(niimg.get_data().dtype, dtype)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/image/resampling.py:442: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()
/network/lustre/dtlake0

>> Subject : hgg_TCIA02_179_1_t1ce


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/_utils/niimg.py:116: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dtype = _get_target_dtype(niimg.get_data().dtype, dtype)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/image/resampling.py:442: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()
/network/lustre/dtlake0

>> Subject : hgg_CBICA_BGW_1_t1ce


/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/_utils/niimg.py:116: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dtype = _get_target_dtype(niimg.get_data().dtype, dtype)
/network/lustre/dtlake01/aramis/users/paul.vernhet/anaconda3/envs/lightning_01/lib/python3.7/site-packages/nilearn/image/resampling.py:442: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()
/network/lustre/dtlake0